In [ ]:
from ccount import *
from os import environ

#todo: faster list comprehension of cropping

## Parameters

# you can adjust these parameters to suit your data
# for run.sh
if environ.get('fname') is not None:
    fname = environ['fname']  # for runipy
else:
    # for notebook running
    fname = 'S0_0MMC_c.czi'

print('fname:', fname)
block_height = 2048 
block_width = 2048 # pixcels
scaling_factor = 4 # 1: original dimension, 2: 1/2, 4: 1/4

In [ ]:
# please don't change unless necessary
blob_extention_ratio = 1.4 # soft extend blob radius manually (1.4)
blob_extention_radius = 2 # pixcels to extend (2)
crop_width = 400  # padding width, which is cropped img width/2 (50)
overlap=.0

if scaling_factor == 1:
    max_sigma=50 
    min_sigma=11
    num_sigma=5
    threshold=0.15
elif scaling_factor == 2:
    max_sigma=40
    min_sigma=8
    num_sigma=5
    threshold=0.15
elif scaling_factor == 4:
    max_sigma=30
    min_sigma=4
    num_sigma=5
    threshold=0.2

    
detected_dbname = re.sub('.czi$', '.npy', fname)
labeled_dbname = re.sub('.czi$', '.labeled.npy', fname)
out_img_fname = re.sub('.czi$', '.jpg', fname)
equ_img_fname = re.sub('.czi$', '.equ.jpg', fname)

## Read whole image (czi)

In [ ]:
print(fname)
image = read_czi(fname)
image = np.divide(image, np.max(image))  # from 0-255 or any range -> 0-1

# image = image[0:2048*3, 0:2048*3]

dims = np.divide(image.shape, 128) # out jpg size: 256 big, 512 mid, 1024 small

## Plot whole image (takes time)

In [ ]:
## Input Image
plt.figure(figsize=(dims[0],dims[1]))
plt.imshow(image, 'gray')
plt.savefig(out_img_fname)

## Equalize whole image

In [ ]:
image_equ = block_equalize(image, block_height=2048, block_width=2048)
image_equ.shape

In [ ]:
## Equalized image
plt.figure(figsize=(dims[0],dims[1]))
plt.imshow(image_equ, 'gray')
plt.savefig(equ_img_fname)

## Find Blobs

In [ ]:
image_flat_crops = find_blobs_and_crop(
    image, image_equ,
    crop_width=crop_width, 
    # blob_detection parameters
    scaling_factor=scaling_factor,
    max_sigma=max_sigma, 
    min_sigma=min_sigma, 
    num_sigma=num_sigma, 
    threshold=threshold, 
    overlap=overlap,
    
    # blob yellow circle visualization parameters
    blob_extention_ratio=blob_extention_ratio,
    blob_extention_radius=blob_extention_radius
)

## Save all blobs detected (for labeling/training)

In [ ]:
print('there are {} blobs detected'.format(len(image_flat_crops)))
print(image_flat_crops.shape)
np.save(detected_dbname, image_flat_crops)
print('saved into {}'.format(detected_dbname))

In [ ]:
## Histogtam of radius
r_ = image_flat_crops[:,2]
plt.hist(r_, 40)
plt.title("Histogram of blob size")
plt.show()

In [ ]:
## show blobs
show_rand_crops(crops=image_flat_crops, label_filter='na', num_shown=30, 
               blob_extention_ratio=blob_extention_ratio, blob_extention_radius=blob_extention_radius)

In [ ]:
## show filtered blobs
filtered_blobs = filter_blobs(blobs=image_flat_crops, r_min=50, r_max=800)
show_rand_crops(crops=filtered_blobs, label_filter='na', num_shown=10,
                blob_extention_ratio=blob_extention_ratio, blob_extention_radius=blob_extention_radius)